# Welcome to the Google Earth Engine (GEE) Python API!

## These notebooks will provide an overview of how to use the GEE python API and access all it has to offer.

# Notebook 3: Visualizing Data using geemap


This notebook covers a quick look into the _geemap_ package, which provides simple python tools for plotting earth engine datasets.
Feel free to check out [geemap.org](http://geemap.org) for more information.

## 1. Creating a map

Creating a map using geemap is simple: Just call _geemap.Map()_

In [1]:
import ee
ee.Initialize()
import geemap
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

This creates an interactive plot with __tons__ of features. Feel free to click around on the above map and explore some of the built-in tools.

You can pass different arguments, including a (lat,lon) point to center the map and a zoom level. Also, the basemap can be changed to any object in the _geemap.basemaps_ dictionary (such as 'HYBRID', 'SATELLITE', or 'NLCD 2019 CONUS Land Cover'):

In [2]:
Map = geemap.Map(center = (40.015,-105),zoom=4,basemap='HYBRID')
Map

Map(center=[40.015, -105], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childre…

## 2. Adding GEE features

As in the last notebook, you can add features from GEE to these maps using _Map.addLayer()_. Let's use the TIGER county database as an example: 

In [3]:
county_name = 'Boulder'
counties = ee.FeatureCollection('TIGER/2018/Counties')
boulderCo = ee.Feature(counties.filter(ee.Filter.eq("NAME", county_name)).first())

In [4]:
Map = geemap.Map(center = (40.015,-105),zoom=6,basemap='HYBRID')
Map.addLayer(boulderCo,{'opacity':.5,'color':'red'},'County Boundary')
Map

Map(center=[40.015, -105], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childre…

The map above can also be set to a center on a given feature:

In [5]:
Map.centerObject(boulderCo, 10)

ee.Feature is called above to get the single feature from the feature collection. However, an entire feature collection can be plotted in the same way. Let's look at all counties in the US named "Jefferson":

In [6]:
county_name = 'Jefferson'
jeffersonCounties = counties.filter(ee.Filter.eq("NAME", county_name))
Map = geemap.Map(center = (40.015,-97.5),zoom=4,basemap='HYBRID')
Map.addLayer(jeffersonCounties,{'opacity':.5,'color':'yellow'},'Jefferson Counties')
Map

Map(center=[40.015, -97.5], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childr…

Quite a few! 

## 3. Adding GEE images

An image can be added in the exact same way. Let's use the landsat images we used in the last notebook. To refresh, these file IDs were:
- LC08_L2SP_033032_20210511_20210524_02_T2
- LC08_L2SP_033033_20210511_20210524_02_T2

Let's use just the first one for now:

In [8]:
landsat_img  = ee.ImageCollection('LANDSAT/LC08/C02/T2_L2')\
                                .filter(ee.Filter.eq('LANDSAT_PRODUCT_ID', 'LC08_L2SP_033032_20210511_20210524_02_T2')).first()
##to refresh, the properties within the image collection on which to filter can be listed with:
col= ee.ImageCollection('LANDSAT/LC08/C02/T2_L2')
col.first().propertyNames().getInfo()
col.first().getInfo()['bands']

[{'id': 'SR_B1',
  'data_type': {'type': 'PixelType',
   'precision': 'int',
   'min': 0,
   'max': 65535},
  'dimensions': [8991, 8971],
  'crs': 'EPSG:32630',
  'crs_transform': [30, 0, 341985, 0, -30, 9016215]},
 {'id': 'SR_B2',
  'data_type': {'type': 'PixelType',
   'precision': 'int',
   'min': 0,
   'max': 65535},
  'dimensions': [8991, 8971],
  'crs': 'EPSG:32630',
  'crs_transform': [30, 0, 341985, 0, -30, 9016215]},
 {'id': 'SR_B3',
  'data_type': {'type': 'PixelType',
   'precision': 'int',
   'min': 0,
   'max': 65535},
  'dimensions': [8991, 8971],
  'crs': 'EPSG:32630',
  'crs_transform': [30, 0, 341985, 0, -30, 9016215]},
 {'id': 'SR_B4',
  'data_type': {'type': 'PixelType',
   'precision': 'int',
   'min': 0,
   'max': 65535},
  'dimensions': [8991, 8971],
  'crs': 'EPSG:32630',
  'crs_transform': [30, 0, 341985, 0, -30, 9016215]},
 {'id': 'SR_B5',
  'data_type': {'type': 'PixelType',
   'precision': 'int',
   'min': 0,
   'max': 65535},
  'dimensions': [8991, 8971],
  

In [9]:
Map = geemap.Map(basemap='HYBRID')
Map.addLayer(landsat_img.select('SR_B7'), {'min': 20000, 'max': 25000}, 'Landsat Band 7')
Map.centerObject(landsat_img.select('SR_B7'), 8)
Map

Map(center=[40.330959838710434, -104.24316276508551], controls=(WidgetControl(options=['position', 'transparen…

Additionally, images can be clipped to the bounds of a feature:

In [10]:
landsat_img_clipped = landsat_img.clip(boulderCo)

In [11]:
Map = geemap.Map(basemap='HYBRID')
Map.centerObject(boulderCo, 10)
vis_params = {
    'color': 'red', 
    'lineType': 'solid',
    'strokeWidth':20,
    'fillColor': None}
Map.addLayer(boulderCo,vis_params,'County Boundary')
Map.addLayer(landsat_img_clipped.select('SR_B1'), {'min': 20000, 'max': 50000}, 'Landsat Band 1')
Map

Map(center=[40.09245085401178, -105.35771122120155], controls=(WidgetControl(options=['position', 'transparent…

## 4. Adding image collections

Instead of adding one image at a time, entire collections can be plotted. For example, let's plot all the landsat images from a given day

In [12]:
ls_collection  = ee.ImageCollection('LANDSAT/LC08/C02/T2_L2').filterDate('2021-07-04', '2021-07-05')
print(str(ls_collection.size().getInfo())+' images in filtered collection')

165 images in filtered collection


In [13]:
Map = geemap.Map(center = (40.015,-105),zoom=2,basemap='HYBRID')
Map.addLayer(ls_collection, {'bands':['SR_B1'],'min':20000,'max':50000}, "Landsat images")
Map

Map(center=[40.015, -105], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childre…

This is one way to visualize the imagery coverage from a given sensor and time range. Additionally, images from other sensors can be added to the same map:

In [14]:
s2_collection  = ee.ImageCollection("COPERNICUS/S2_SR").filterDate('2021-07-04T00:00', '2021-07-04T02:00')
print(str(s2_collection.size().getInfo())+' images in filtered collection')

883 images in filtered collection


In [15]:
Map.addLayer(s2_collection, {'bands':['B1'],'min':0,'max':2000}, "Sentinel-2 images")

This was a simple overview on how to add images / features to a map using geemap. However, these are just single bands that don't provide as much information as they could.
In the next notebook, more advanced plotting techniques will be covered.
